In [132]:
import torch
print(torch.__version__)

1.8.1+cu101


In [175]:
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import nn, optim
import matplotlib.pyplot as plt
from time import time
import numpy as np
import pandas as pd
from six.moves import urllib
import random
from skimage.util import random_noise
from math import log10
import torch.nn.functional as F
import math

In [134]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [135]:
!nvidia-smi

Mon Apr 19 01:19:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    39W / 300W |  12853MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [136]:
# Read the train and test sets of CIFAR-10 data
default_transform = transforms.Compose([transforms.ToTensor()])
cifar_trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=default_transform)
cifar_testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=default_transform)
print("Training set size:", len(cifar_trainset))
print("Test set size:", len(cifar_testset))

Files already downloaded and verified
Files already downloaded and verified
Training set size: 50000
Test set size: 10000


In [137]:
# Initialize data loader functions
BATCH_SIZE = 64
train_dataLoader = DataLoader(cifar_trainset, batch_size=BATCH_SIZE, shuffle=True)
test_dataLoader = DataLoader(cifar_testset, batch_size=BATCH_SIZE, shuffle=False)

In [138]:
# Validate shape of the input images
dataiter = iter(train_dataLoader)
images, labels = dataiter.next()

print(images.shape)
print(labels.shape)

torch.Size([64, 3, 32, 32])
torch.Size([64])


In [162]:
def conv1x1(in_channels, out_channels, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

In [163]:
def conv3x3(in_channels, out_channels, stride=1, groups=1, dilation=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, groups=groups, bias=False)

In [164]:
class ResidualBlock(nn.Module):
    expansion: int = 1
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        # print(out.shape)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [165]:
class Bottleneck(nn.Module):
    expansion: int = 4
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None, groups = 1, base_width = 16, dilation = 1, norm_layer = None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        
        width = int(out_channels * (base_width / 16.)) * groups
       
        self.conv1 = conv1x1(in_channels, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, out_channels * self.expansion)
        self.bn3 = norm_layer(out_channels * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        # print(out.shape)
        # out = out.view(-1, 64*64*8*8)
        # print(out.shape)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        
        return out


        # flatten the output of conv layers


In [166]:
# ResNet (Referenced from Pytorch official tutorial)
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()

        self.in_channels = 16

        self.conv = nn.Conv2d(3, self.in_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(self.in_channels)

        self.layer1 = self.make_layer(block, 16, layers[0], stride=1)
        self.layer2 = self.make_layer(block, 32, layers[1], stride=2)
        self.layer3 = self.make_layer(block, 64, layers[2], stride=2)
       
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if ((stride != 1) or (self.in_channels != out_channels * block.expansion)):
            downsample = nn.Sequential(
                conv1x1(self.in_channels, out_channels*block.expansion, stride=stride),
                nn.BatchNorm2d(out_channels*block.expansion))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = F.relu(out)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        print("3rd layer out:", out.shape)

        out = F.avg_pool2d(out, out.size()[3])
        print("Avg. pool:", out.shape)
        # out = out.view(out.size(0), -1)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        
        return out

In [172]:
### ALTERNATE
class Bottleneck_alt(nn.Module):
    expansion = 4  # # output cahnnels / # input channels

    def __init__(self, inplanes, outplanes, stride=1):
        assert outplanes % self.expansion == 0
        super(Bottleneck_alt, self).__init__()
        self.inplanes = inplanes
        self.outplanes = outplanes
        self.bottleneck_planes = int(outplanes / self.expansion)
        self.stride = stride

        self._make_layer()

    def _make_layer(self):
        # conv 1x1
        self.bn1 = nn.BatchNorm2d(self.inplanes)
        self.conv1 = nn.Conv2d(self.inplanes, self.bottleneck_planes,
                               kernel_size=1, stride=self.stride, bias=False)
        # conv 3x3
        self.bn2 = nn.BatchNorm2d(self.bottleneck_planes)
        self.conv2 = nn.Conv2d(self.bottleneck_planes, self.bottleneck_planes,
                               kernel_size=3, stride=1, padding=1, bias=False)
        # conv 1x1
        self.bn3 = nn.BatchNorm2d(self.bottleneck_planes)
        self.conv3 = nn.Conv2d(self.bottleneck_planes, self.outplanes, kernel_size=1,
                               stride=1)
        if self.inplanes != self.outplanes:
            self.shortcut = nn.Conv2d(self.inplanes, self.outplanes, kernel_size=1,
                                      stride=self.stride, bias=False)
        else:
            self.shortcut = None
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        residual = x
        # we do pre-activation
        out = self.relu(self.bn1(x))
        out = self.conv1(out)

        out = self.relu(self.bn2(out))
        out = self.conv2(out)

        out = self.relu(self.bn3(out))
        out = self.conv3(out)

        if self.shortcut is not None:
            residual = self.shortcut(residual)

        out += residual
        return out

In [173]:
## ALTERNATE
class ResNet_alt(nn.Module):
    def __init__(self, block, depth, output_classes=10):
        assert (depth - 2) % 9 == 0  # 164 or 1001
        super(ResNet_alt, self).__init__()
        n = int((depth - 2) / 9)
        nstages = [16, 64, 128, 256]
        # one conv at the beginning (spatial size: 32x32)
        self.conv1 = nn.Conv2d(3, nstages[0], kernel_size=3, stride=1,
                               padding=1, bias=False)

        # use `block` as unit to construct res-net
        # Stage 0 (spatial size: 32x32)
        self.layer1 = self._make_layer(block, nstages[0], nstages[1], n)
        # Stage 1 (spatial size: 32x32)
        self.layer2 = self._make_layer(block, nstages[1], nstages[2], n, stride=2)
        # Stage 2 (spatial size: 16x16)
        self.layer3 = self._make_layer(block, nstages[2], nstages[3], n, stride=2)
        # Stage 3 (spatial size: 8x8)
        self.bn = nn.BatchNorm2d(nstages[3])
        self.relu = nn.ReLU(inplace=True)
        # classifier
        self.avgpool = nn.AvgPool2d(8)
        self.fc = nn.Linear(nstages[3], output_classes)

        # weight initialization
        self._init_weights()

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, inplanes, outplanes, nstage, stride=1):
        layers = []
        layers.append(block(inplanes, outplanes, stride))
        for i in range(1, nstage):
            layers.append(block(outplanes, outplanes, stride=1))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.relu(self.bn(x))

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [ ]:
def train_network(model, num_epochs, learning_rate, train_dataLoader, test_dataLoader, lr_update_rule):
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  train_accuracy_list, test_accuracy_list, train_loss_list = [], [], [] 
  for epoch in range(num_epochs):
    running_loss = 0
    for images, labels in train_dataLoader:
      imgs = images.to(device)
      lbls = labels.to(device)
  
      # Training step
      optimizer.zero_grad()
      out = model(imgs)
      loss = criterion(out, lbls)
      
      # Backpropagate loss
      loss.backward()
      
      # Optimize weights
      optimizer.step()
      
      running_loss += loss.item()
    
    train_loss = running_loss/len(train_dataLoader)
    train_loss_list.append(train_loss)
    train_accuracy = get_accuracy(model, train_dataLoader)
    train_accuracy_list.append(train_accuracy)
    
    test_accuracy = get_accuracy(model, test_dataLoader)
    test_accuracy_list.append(test_accuracy)
    print("Epoch: {} \t Training loss: {} \t Training accuracy: {} \t Test accuracy: {}".format(epoch, train_loss, train_accuracy, test_accuracy))
    
  return model, train_accuracy_list, test_accuracy_list, train_loss_list

In [ ]:
def get_accuracy(model, dataloader):
  """
    Function to compute accuracy given a model (of class nn) and a dataloader object
  """
  
  model.eval()
  correct_predictions = 0
  with torch.no_grad():
    for images, labels in dataloader:
      imgs = images.to(device)
      lbls = labels.to(device)
      # images = images.view(images.shape[0], -1)
      output = model(imgs)
      _, predicted = torch.max(output.data, 1)
      correct_predictions += (predicted == lbls).sum().item()
  accuracy = (correct_predictions / len(dataloader.dataset)) * 100
  return(accuracy)


In [189]:
# Hyper-parameters
EPOCHS = 5
lr = 0.001
lr_update = {80:0.0001, 120:0.00001, 160:0.000001}

In [177]:
resnet110B = ResNet_alt(Bottleneck_alt, 110, 10).to(device)
resnet110B, train_acc_110B, test_acc_110B, train_loss_110B = train_network(model=resnet110B,
                                                                       num_epochs=EPOCHS,
                                                                       learning_rate=lr,
                                                                       train_dataLoader=train_dataLoader,
                                                                       test_dataLoader=test_dataLoader,
                                                                       lr_update_rule=lr_update)

Epoch: 0 	 Training loss: 1.4181287358026675 	 Training accuracy: 50.70400000000001 	 Test accuracy: 49.13
Epoch: 1 	 Training loss: 1.2075940660198632 	 Training accuracy: 62.470000000000006 	 Test accuracy: 60.85
Epoch: 2 	 Training loss: 0.9436029142430981 	 Training accuracy: 69.77799999999999 	 Test accuracy: 67.56
Epoch: 3 	 Training loss: 0.8139959624432542 	 Training accuracy: 73.978 	 Test accuracy: 70.38
Epoch: 4 	 Training loss: 0.7255502210172546 	 Training accuracy: 76.94 	 Test accuracy: 72.78
Epoch: 5 	 Training loss: 0.635001904252545 	 Training accuracy: 79.386 	 Test accuracy: 73.99
Epoch: 6 	 Training loss: 0.5627009809550727 	 Training accuracy: 84.408 	 Test accuracy: 77.58
Epoch: 7 	 Training loss: 0.5035554345344644 	 Training accuracy: 85.994 	 Test accuracy: 78.03999999999999
Epoch: 8 	 Training loss: 0.4345067731674065 	 Training accuracy: 88.39399999999999 	 Test accuracy: 79.19
Epoch: 9 	 Training loss: 0.3904125106227977 	 Training accuracy: 89.0 	 Test acc

In [178]:
import pandas as pd
metric_data = pd.DataFrame({'Epoch': range(1,EPOCHS+1), 'Train_Acc': train_acc_110B, 'Test_Acc': test_acc_110B, 'Train_Loss': train_loss_110B})
metric_data.to_csv('ResNet110_B.csv', index=False)

In [180]:
torch.save(resnet110B, 'ResNet110_B.pt')

## IC Layer - Variant 1 (IC --> Conv2D --> ReLU)

In [182]:
def IC(inputs, p=0.03):
  y = nn.Sequential(
      nn.BatchNorm2d(inputs),
      nn.Dropout(p))

  # y = nn.Dropout(p)(y)

  return y

In [185]:
class Bottleneck_IC1(nn.Module):
    expansion = 4  # # output cahnnels / # input channels

    def __init__(self, inplanes, outplanes, stride=1):
        assert outplanes % self.expansion == 0
        super(Bottleneck_IC1, self).__init__()
        self.inplanes = inplanes
        self.outplanes = outplanes
        self.bottleneck_planes = int(outplanes / self.expansion)
        self.stride = stride

        self._make_layer()

    def _make_layer(self):
        # conv 1x1
        # self.bn1 = nn.BatchNorm2d(self.inplanes)
        self.IC1 = IC(self.inplanes)
        self.conv1 = nn.Conv2d(self.inplanes, self.bottleneck_planes,
                               kernel_size=1, stride=self.stride, bias=False)
        # conv 3x3
        # self.bn2 = nn.BatchNorm2d(self.bottleneck_planes)
        self.IC2 = IC(self.bottleneck_planes)
        self.conv2 = nn.Conv2d(self.bottleneck_planes, self.bottleneck_planes,
                               kernel_size=3, stride=1, padding=1, bias=False)
        # conv 1x1
        # self.bn3 = nn.BatchNorm2d(self.bottleneck_planes)
        self.IC3 = IC(self.bottleneck_planes)
        self.conv3 = nn.Conv2d(self.bottleneck_planes, self.outplanes, kernel_size=1,
                               stride=1)
        if self.inplanes != self.outplanes:
            self.shortcut = nn.Conv2d(self.inplanes, self.outplanes, kernel_size=1,
                                      stride=self.stride, bias=False)
        else:
            self.shortcut = None
        self.relu = nn.ReLU()

    def forward(self, x):
        residual = x
  
        out = self.IC1(x)
        out = self.conv1(out)
        out = self.relu(out)

        out = self.IC2(out)
        out = self.conv2(out)
        out = self.relu(out)

        out = self.IC3(out)
        out = self.conv3(out)
        out = self.relu(out)

        if self.shortcut is not None:
            residual = self.shortcut(residual)

        out += residual
        return out

In [186]:
resnet110B_ic1 = ResNet_alt(Bottleneck_IC1, 110, 10).to(device)

resnet110B_ic1, train_acc_110B_ic1, test_acc_110B_ic1, train_loss_110B_ic1 = train_network(model=resnet110B_ic1,
                                                                       num_epochs=EPOCHS,
                                                                       learning_rate=lr,
                                                                       train_dataLoader=train_dataLoader,
                                                                       test_dataLoader=test_dataLoader,
                                                                       lr_update_rule=lr_update)

Epoch: 0 	 Training loss: 1.4386122006437052 	 Training accuracy: 53.834 	 Test accuracy: 52.78
Epoch: 1 	 Training loss: 1.1228865907930048 	 Training accuracy: 67.776 	 Test accuracy: 65.91
Epoch: 2 	 Training loss: 0.881780085737443 	 Training accuracy: 69.66799999999999 	 Test accuracy: 66.52
Epoch: 3 	 Training loss: 0.770281089007702 	 Training accuracy: 75.144 	 Test accuracy: 70.81
Epoch: 4 	 Training loss: 0.6609111375477917 	 Training accuracy: 78.742 	 Test accuracy: 74.06
Epoch: 5 	 Training loss: 0.5650976640755868 	 Training accuracy: 83.37599999999999 	 Test accuracy: 77.41
Epoch: 6 	 Training loss: 0.5030645015256484 	 Training accuracy: 86.76 	 Test accuracy: 79.02
Epoch: 7 	 Training loss: 0.4299528250647018 	 Training accuracy: 86.71 	 Test accuracy: 78.99000000000001
Epoch: 8 	 Training loss: 0.3808275396313966 	 Training accuracy: 86.53 	 Test accuracy: 77.12
Epoch: 9 	 Training loss: 0.3354267546778445 	 Training accuracy: 91.01 	 Test accuracy: 79.75999999999999


In [187]:
import pandas as pd
metric_data = pd.DataFrame({'Epoch': range(1,EPOCHS+1), 'Train_Acc': train_acc_110B_ic1, 'Test_Acc': test_acc_110B_ic1, 'Train_Loss': train_loss_110B_ic1})
metric_data.to_csv('ResNet110_B_ic1.csv', index=False)

In [188]:
torch.save(resnet110B_ic1, 'ResNet110_B_ic1.pt')

## Variant 2 (Conv2D --> ReLU --> IC)

In [ ]:
class ResidualBlock_IC_2(nn.Module):
    expansion: int = 1
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock_IC_2, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.relu = nn.ReLU(inplace=True)
        self.IC = IC(out_channels)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.relu(out)
        out = self.IC(out)
        out = self.conv2(out)
        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

In [190]:
class Bottleneck_IC2(nn.Module):
    expansion = 4  # # output cahnnels / # input channels

    def __init__(self, inplanes, outplanes, stride=1):
        assert outplanes % self.expansion == 0
        super(Bottleneck_IC2, self).__init__()
        self.inplanes = inplanes
        self.outplanes = outplanes
        self.bottleneck_planes = int(outplanes / self.expansion)
        self.stride = stride

        self._make_layer()

    def _make_layer(self):
        # conv 1x1
        # self.bn1 = nn.BatchNorm2d(self.inplanes)
        
        self.conv1 = nn.Conv2d(self.inplanes, self.bottleneck_planes,
                               kernel_size=1, stride=self.stride, bias=False)
        self.IC1 = IC(self.bottleneck_planes)
        # conv 3x3
        # self.bn2 = nn.BatchNorm2d(self.bottleneck_planes)
        
        self.conv2 = nn.Conv2d(self.bottleneck_planes, self.bottleneck_planes,
                               kernel_size=3, stride=1, padding=1, bias=False)
        self.IC2 = IC(self.bottleneck_planes)
        # conv 1x1
        # self.bn3 = nn.BatchNorm2d(self.bottleneck_planes)
        
        self.conv3 = nn.Conv2d(self.bottleneck_planes, self.outplanes, kernel_size=1,
                               stride=1)
        self.IC3 = IC(self.outplanes)

        if self.inplanes != self.outplanes:
            self.shortcut = nn.Conv2d(self.inplanes, self.outplanes, kernel_size=1,
                                      stride=self.stride, bias=False)
        else:
            self.shortcut = None
        self.relu = nn.ReLU()

    def forward(self, x):
        residual = x
  
        out = self.conv1(x)
        out = self.relu(out)
        out = self.IC1(out)

        out = self.conv2(out)
        out = self.relu(out)
        out = self.IC2(out)

        out = self.conv3(out)
        out = self.relu(out)
        out = self.IC3(out)

        if self.shortcut is not None:
            residual = self.shortcut(residual)

        out += residual
        return out

In [193]:
resnet110B_ic2 = ResNet_alt(Bottleneck_IC2, 110,10).to(device)
resnet110B_ic2, train_acc_110B_ic2, test_acc_110B_ic2, train_loss_110B_ic2 = train_network(model=resnet110B_ic2,
                                                                       num_epochs=EPOCHS,
                                                                       learning_rate=lr,
                                                                       train_dataLoader=train_dataLoader,
                                                                       test_dataLoader=test_dataLoader,
                                                                       lr_update_rule=lr_update)

Epoch: 0 	 Training loss: 1.4796148355659622 	 Training accuracy: 57.548 	 Test accuracy: 56.71000000000001
Epoch: 1 	 Training loss: 1.0957057612478887 	 Training accuracy: 67.25999999999999 	 Test accuracy: 65.14999999999999
Epoch: 2 	 Training loss: 0.8843154072609094 	 Training accuracy: 69.96 	 Test accuracy: 67.39
Epoch: 3 	 Training loss: 0.7481784776348592 	 Training accuracy: 76.59 	 Test accuracy: 72.85000000000001
Epoch: 4 	 Training loss: 0.6352573958656672 	 Training accuracy: 81.478 	 Test accuracy: 75.91


In [ ]:
metric_data = pd.DataFrame({'Epoch': range(1,EPOCHS+1), 'Train_Acc': train_acc_110B_ic2, 'Test_Acc': test_acc_110B_ic2, 'Train_Loss': train_loss_110B_ic2})
metric_data.to_csv('ResNet110_B_ic2.csv', index=False)
torch.save(resnet110B_ic2, 'ResNet110_B_ic2.pt')

## Variant 3 (ReLU --> IC --> Conv@D)

In [ ]:
class ResidualBlock_IC_3(nn.Module):
    expansion: int = 1
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock_IC_3, self).__init__()
        self.relu = nn.ReLU(inplace=True)
        self.IC = IC(in_channels)
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.relu(x)
        out = self.IC(out)
        out = self.conv1(out)
        out = self.conv2(out)
        if self.downsample is not None:
            residual = self.downsample(x)
        out += residual
        # out = self.relu(out)
        return out

In [196]:
class Bottleneck_IC3(nn.Module):
    expansion = 4  # # output cahnnels / # input channels

    def __init__(self, inplanes, outplanes, stride=1):
        assert outplanes % self.expansion == 0
        super(Bottleneck_IC3, self).__init__()
        self.inplanes = inplanes
        self.outplanes = outplanes
        self.bottleneck_planes = int(outplanes / self.expansion)
        self.stride = stride

        self._make_layer()

    def _make_layer(self):
        # conv 1x1
        # self.bn1 = nn.BatchNorm2d(self.inplanes)
        self.IC1 = IC(self.inplanes)
        self.conv1 = nn.Conv2d(self.inplanes, self.bottleneck_planes,
                               kernel_size=1, stride=self.stride, bias=False)
        
        # conv 3x3
        # self.bn2 = nn.BatchNorm2d(self.bottleneck_planes)
        self.IC2 = IC(self.bottleneck_planes)
        self.conv2 = nn.Conv2d(self.bottleneck_planes, self.bottleneck_planes,
                               kernel_size=3, stride=1, padding=1, bias=False)
        
        # conv 1x1
        # self.bn3 = nn.BatchNorm2d(self.bottleneck_planes)
        self.IC3 = IC(self.bottleneck_planes)
        self.conv3 = nn.Conv2d(self.bottleneck_planes, self.outplanes, kernel_size=1,
                               stride=1)
        

        if self.inplanes != self.outplanes:
            self.shortcut = nn.Conv2d(self.inplanes, self.outplanes, kernel_size=1,
                                      stride=self.stride, bias=False)
        else:
            self.shortcut = None
        self.relu = nn.ReLU()

    def forward(self, x):
        residual = x
  
        out = self.relu(x)
        out = self.IC1(out)
        out = self.conv1(out)

        out = self.relu(out)
        out = self.IC2(out)
        out = self.conv2(out)
        

        out = self.relu(out)
        out = self.IC3(out)
        out = self.conv3(out)
        
        if self.shortcut is not None:
            residual = self.shortcut(residual)

        out += residual
        return out

In [197]:
resnet110B_ic3 = ResNet_alt(Bottleneck_IC3, 110,10).to(device)
resnet110B_ic3, train_acc_110B_ic3, test_acc_110B_ic3, train_loss_110B_ic3 = train_network(model=resnet110B_ic3,
                                                                       num_epochs=EPOCHS,
                                                                       learning_rate=lr,
                                                                       train_dataLoader=train_dataLoader,
                                                                       test_dataLoader=test_dataLoader,
                                                                       lr_update_rule=lr_update)

Epoch: 0 	 Training loss: 1.4995829625355312 	 Training accuracy: 55.876000000000005 	 Test accuracy: 54.71
Epoch: 1 	 Training loss: 1.1024252461350483 	 Training accuracy: 64.97800000000001 	 Test accuracy: 62.69
Epoch: 2 	 Training loss: 0.864441360582781 	 Training accuracy: 74.78399999999999 	 Test accuracy: 71.17999999999999
Epoch: 3 	 Training loss: 0.7156884879864696 	 Training accuracy: 78.69 	 Test accuracy: 73.67
Epoch: 4 	 Training loss: 0.5986189532386678 	 Training accuracy: 80.67999999999999 	 Test accuracy: 74.5


In [198]:
metric_data_ic3 = pd.DataFrame({'Epoch': range(1,EPOCHS+1), 'Train_Acc': train_acc_110B_ic3, 
                                'Test_Acc': test_acc_110B_ic3, 'Train_Loss': train_loss_110B_ic3})
metric_data_ic3.to_csv('ResNet110_B_ic3.csv', index=False)
torch.save(resnet110B_ic3, 'ResNet110_B_ic3.pt')

For ResNet 164

In [199]:
resnet164B = ResNet_alt(Bottleneck_alt, 164, 10).to(device)
resnet164B, train_acc_164B, test_acc_164B, train_loss_164B = train_network(model=resnet164B,
                                                                       num_epochs=EPOCHS,
                                                                       learning_rate=lr,
                                                                       train_dataLoader=train_dataLoader,
                                                                       test_dataLoader=test_dataLoader,
                                                                       lr_update_rule=lr_update)

Epoch: 0 	 Training loss: 1.459806117712689 	 Training accuracy: 56.218 	 Test accuracy: 55.25
Epoch: 1 	 Training loss: 1.4221299736548567 	 Training accuracy: 55.86600000000001 	 Test accuracy: 54.49
Epoch: 2 	 Training loss: 1.054885174612255 	 Training accuracy: 68.312 	 Test accuracy: 66.25
Epoch: 3 	 Training loss: 0.8999952042041837 	 Training accuracy: 66.778 	 Test accuracy: 64.2
Epoch: 4 	 Training loss: 0.7872910140572912 	 Training accuracy: 75.254 	 Test accuracy: 71.87


In [200]:
EPOCHS =3

In [201]:
resnet164B_ic1 = ResNet_alt(Bottleneck_IC1, 164, 10).to(device)

resnet164B_ic1, train_acc_164B_ic1, test_acc_164B_ic1, train_loss_164B_ic1 = train_network(model=resnet164B_ic1,
                                                                       num_epochs=EPOCHS,
                                                                       learning_rate=lr,
                                                                       train_dataLoader=train_dataLoader,
                                                                       test_dataLoader=test_dataLoader,
                                                                       lr_update_rule=lr_update)

Epoch: 0 	 Training loss: 1.5166170251034106 	 Training accuracy: 51.361999999999995 	 Test accuracy: 50.77
Epoch: 1 	 Training loss: 1.2802430395884892 	 Training accuracy: 61.104 	 Test accuracy: 59.14
Epoch: 2 	 Training loss: 0.9446688065748385 	 Training accuracy: 71.414 	 Test accuracy: 68.39


In [ ]:
resnet164B_ic2 = ResNet_alt(Bottleneck_IC2, 164,10).to(device)
resnet164B_ic2, train_acc_164B_ic2, test_acc_164B_ic2, train_loss_164B_ic2 = train_network(model=resnet164B_ic2,
                                                                       num_epochs=EPOCHS,
                                                                       learning_rate=lr,
                                                                       train_dataLoader=train_dataLoader,
                                                                       test_dataLoader=test_dataLoader,
                                                                       lr_update_rule=lr_update)

In [ ]:
resnet164B_ic3 = ResNet_alt(Bottleneck_IC3, 164,10).to(device)
resnet164B_ic3, train_acc_164B_ic3, test_acc_164B_ic3, train_loss_164B_ic3 = train_network(model=resnet164B_ic3,
                                                                       num_epochs=EPOCHS,
                                                                       learning_rate=lr,
                                                                       train_dataLoader=train_dataLoader,
                                                                       test_dataLoader=test_dataLoader,
                                                                       lr_update_rule=lr_update)